In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import csv
import numpy as np
import pandas as pd

# Data Setup

In [87]:
all_data = dict()

for time in [1, 2, 3, 4]:
    for ver in ["tr", "test"]:
        filename = f"{time}_{ver}"
        data = pd.read_csv(f"{filename}.csv")
        all_data[filename] = dict()
        all_data[filename]["X"] = data.drop(columns=["warstds"])
        all_data[filename]["y"] = data["warstds"]

FileNotFoundError: [Errno 2] File 1_tr.csv does not exist: '1_tr.csv'

# Exp 1: train on > 1965, test on 1945-65

In [83]:
train_X = pd.concat(
    [all_data[f"{all_times[1]}_tr"]["X"],
    all_data[f"{all_times[1]}_test"]["X"],
    all_data[f"{all_times[2]}_tr"]["X"],
    all_data[f"{all_times[2]}_test"]["X"]])

train_y = pd.concat(
    [all_data[f"{all_times[1]}_tr"]["y"],
    all_data[f"{all_times[1]}_test"]["y"],
    all_data[f"{all_times[2]}_tr"]["y"],
    all_data[f"{all_times[2]}_test"]["y"]])

test_X = pd.concat(
    [all_data[f"{all_times[0]}_tr"]["X"],
    all_data[f"{all_times[0]}_test"]["X"]])

test_y = pd.concat(
    [all_data[f"{all_times[0]}_tr"]["y"],
    all_data[f"{all_times[0]}_test"]["y"]])

log_reg = LogisticRegression().fit(train_X, train_y)
log_reg_score = log_reg.score(test_X, test_y)
rdm_for = RandomForestClassifier(max_depth=5).fit(train_X, train_y)
rdm_for_score = rdm_for.score(test_X, test_y)
print("Training Scores for Logistic Regression: {0}, Random Forest: {1}\n".format(log_reg_score, rdm_for_score))

Training Scores for Logistic Regression: 0.9834733893557422, Random Forest: 0.9955182072829132



# Exp 2: Testing "in the middle"

In [84]:
train_X = pd.concat(
    [all_data[f"{all_times[0]}_tr"]["X"],
    all_data[f"{all_times[0]}_test"]["X"],
    all_data[f"{all_times[2]}_tr"]["X"],
    all_data[f"{all_times[2]}_test"]["X"]])

train_y = pd.concat(
    [all_data[f"{all_times[0]}_tr"]["y"],
    all_data[f"{all_times[0]}_test"]["y"],
    all_data[f"{all_times[2]}_tr"]["y"],
    all_data[f"{all_times[2]}_test"]["y"]])

test_X = pd.concat(
    [all_data[f"{all_times[1]}_tr"]["X"],
    all_data[f"{all_times[1]}_test"]["X"]])

test_y = pd.concat(
    [all_data[f"{all_times[1]}_tr"]["y"],
    all_data[f"{all_times[1]}_test"]["y"]])

log_reg = LogisticRegression().fit(train_X, train_y)
log_reg_score = log_reg.score(test_X, test_y)
rdm_for = RandomForestClassifier(max_depth=5).fit(train_X, train_y)
rdm_for_score = rdm_for.score(test_X, test_y)
print("Training Scores for Logistic Regression: {0}, Random Forest: {1}\n".format(log_reg_score, rdm_for_score))

Training Scores for Logistic Regression: 0.9836134453781512, Random Forest: 0.9949579831932773



# Exp 3: Train on < 1985, test on 1985-2000

In [85]:
train_X = pd.concat(
    [all_data[f"{all_times[0]}_tr"]["X"],
    all_data[f"{all_times[0]}_test"]["X"],
    all_data[f"{all_times[1]}_tr"]["X"],
    all_data[f"{all_times[1]}_test"]["X"]])

train_y = pd.concat(
    [all_data[f"{all_times[0]}_tr"]["y"],
    all_data[f"{all_times[0]}_test"]["y"],
    all_data[f"{all_times[1]}_tr"]["y"],
    all_data[f"{all_times[1]}_test"]["y"]])

test_X = pd.concat(
    [all_data[f"{all_times[2]}_tr"]["X"],
    all_data[f"{all_times[2]}_test"]["X"]])

test_y = pd.concat(
    [all_data[f"{all_times[2]}_tr"]["y"],
    all_data[f"{all_times[2]}_test"]["y"]])

log_reg = LogisticRegression().fit(train_X, train_y)
log_reg_score = log_reg.score(test_X, test_y)
rdm_for = RandomForestClassifier(max_depth=5).fit(train_X, train_y)
rdm_for_score = rdm_for.score(test_X, test_y)
print("Training Scores for Logistic Regression: {0}, Random Forest: {1}\n".format(log_reg_score, rdm_for_score))

Training Scores for Logistic Regression: 0.9837535014005602, Random Forest: 0.9952380952380953



In [86]:
all_data["1945-65_tr"]["X"]

,cowcode,year,ptime,yrint,autonomy,rf,popdense,auto98,dem98,pol98,...,decade1,decade2,decade3,decade4,independ,tip,anocracy,proxregc,sxpnew.2,sxpsq.2
0,700.0,1965.0,252.0,20.0,0.0,1.000000,17.045191,7.000000,0.00000,-7.000000,...,1.0,0.0,0.0,0.0,1.0,37.000000,0.0,1.000000e+00,0.085721,0.085721
1,700.0,1966.0,264.0,21.0,0.0,1.000000,17.429300,7.000000,0.00000,-7.000000,...,1.0,0.0,0.0,0.0,1.0,38.000000,0.0,2.500000e-01,-0.738777,-0.738777
2,700.0,1967.0,276.0,22.0,0.0,1.000000,17.827311,7.000000,0.00000,-7.000000,...,1.0,0.0,0.0,0.0,1.0,39.000000,0.0,6.250000e-02,-0.959431,-0.959431
3,700.0,1968.0,288.0,23.0,0.0,1.000000,18.238970,7.000000,0.00000,-7.000000,...,1.0,0.0,0.0,0.0,1.0,40.000000,0.0,1.562500e-02,-1.004694,-1.004694
4,700.0,1969.0,300.0,24.0,0.0,1.000000,18.664190,7.000000,0.00000,-7.000000,...,1.0,0.0,0.0,0.0,1.0,41.000000,0.0,3.906250e-03,-0.989380,-0.989380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5392,552.0,1996.0,280.0,30.0,0.0,59.000000,29.060539,6.000000,0.00000,-6.000000,...,0.0,0.0,0.0,1.0,1.0,8.000000,0.0,1.530000e-05,0.259141,0.259141
5393,552.0,1997.0,292.0,31.0,0.0,59.000000,118.015104,6.000000,0.00000,-6.000000,...,0.0,0.0,0.0,1.0,1.0,9.000000,0.0,3.810000e-06,0.452436,0.452436
5394,552.0,1998.0,304.0,32.0,0.0,59.000000,117.788461,5.000000,0.00000,-5.000000,...,0.0,0.0,0.0,1.0,1.0,10.000000,0.0,9.540000e-07,0.097356,0.097356
5395,552.0,1999.0,316.0,33.0,0.0,59.000000,119.010582,3.962030,3.97096,0.008930,...,0.0,0.0,0.0,1.0,1.0,11.000000,0.0,2.380000e-07,0.090076,0.090076
